# Open-Meteo Weather Data

This notebook downloads all historical weather data for Gävle.

In [1]:
import datetime
import feature_store as fs
import os
import openmeteo_requests
import pandas as pd

/Users/ph/Programming/KTH/ID2223Project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Note: currently not needed.
om_key = open(".openmeteo_key", "r").read().strip()

In [3]:
genesis = "2023-01-04"
today_dt = datetime.datetime.today()
today = today_dt.strftime("%Y-%m-%d")

longitude = 60.6749
latitude = 17.1413
dl_dir = os.path.join("dev_data", "openmeteo_download")

In [4]:
om_client = openmeteo_requests.Client()

In [5]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": longitude,
	"longitude": latitude,
	"start_date": genesis,
	"end_date": today,
	"hourly": ["temperature_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "cloud_cover", "wind_speed_10m", "wind_speed_100m", "wind_gusts_10m"]
}
responses = om_client.weather_api(url, params=params)

In [6]:
responses

In [7]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 60.70298767089844°N 17.10213851928711°E
Elevation 17.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [8]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(5).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(9).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.dropna(inplace=True)
fs.upsert_weather(hourly_dataframe)

In [9]:
fs.load_weather()

,date,temperature_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_speed_100m,wind_gusts_10m
0,2023-01-04 00:00:00+00:00,-12.289499,-17.435226,0.0,0.0,0.0,0.07,0.0,11.298495,15.294703,13.679999
1,2023-01-04 01:00:00+00:00,-9.039499,-13.314471,0.0,0.0,0.0,0.07,0.0,6.763786,14.799459,13.320000
2,2023-01-04 02:00:00+00:00,-9.289499,-13.634878,0.0,0.0,0.0,0.07,0.0,7.091177,14.345898,11.159999
3,2023-01-04 03:00:00+00:00,-9.239500,-13.778955,0.0,0.0,0.0,0.07,0.0,8.396570,17.280001,12.599999
4,2023-01-04 04:00:00+00:00,-8.489500,-13.140070,0.0,0.0,0.0,0.07,68.0,9.605998,19.652176,16.559999
...,...,...,...,...,...,...,...,...,...,...,...
16795,2024-12-03 19:00:00+00:00,-4.889500,-9.184601,0.0,0.0,0.0,0.03,0.0,8.274128,21.392754,17.639999
16796,2024-12-03 20:00:00+00:00,-5.089500,-9.354580,0.0,0.0,0.0,0.03,0.0,8.067936,20.215368,15.119999
16797,2024-12-03 21:00:00+00:00,-5.139500,-9.408211,0.0,0.0,0.0,0.03,0.0,8.217153,18.892282,15.480000
16798,2024-12-03 22:00:00+00:00,-5.589500,-9.695892,0.0,0.0,0.0,0.03,0.0,6.948093,18.130030,15.119999
